In [1]:
from utils import read_json_path, query_route
import googlemaps
import pandas as pd

GOOGLE_API_KEY = open('google_maps_key', 'r').readlines()[0]
MAPBOX_API_KEY = open('mapbox_key', 'r').readlines()[0]

gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

# Idea Backlog
* If we query mapbox for all routes at the start of the hour, we'll have really bad performance at the start then good performance later on; what if we iteratively query throughout the hour, while using historical data for the rest
* Moreover, we shouldn't query ALL jeeps to ALL stops in the route, cuz that will cost lots of money
    * What we should do instead is figure out which stop it's closest to, and query that

Hint: The database table needs to look like this

|StopID|List of Jeeps|List of Arrival Times|List of Jeep Routes|
|-----|-----|-----|-----|
|AL1|ID1, ID2|5.5, 10|Aurora Loop, Aurora Loop|
|AL2|ID3, ID1|6, 20|Aurora Loop, Aurora Loop|
|QA1|ID4, ID5|10, 20|Quezon Ave Loop, Quezon Ave Loop|
|QA2|ID6, ID4|10, 20|Quezon Ave Loop, Quezon Ave Loop|

In [2]:
trinoma = {"lat": 14.653053, "lng": 121.034895}
lung_center = {"lat": 14.647911, "lng": 121.046710}
sachi = {"lat": 14.646185, "lng": 121.052834}
    
stop_coords_mapping_dict = read_json_path("data/stop_coords_mapping_dict.json")
jeep_route_mapping_dict = read_json_path("data/jeep_route_mapping_dict.json")
route_stops_mapping_dict = read_json_path("data/route_stops_mapping_dict.json")


In [3]:
route_name = "aurora_loop"

In [4]:
temp = query_route("aurora_loop", jeep_route_mapping_dict, route_stops_mapping_dict, stop_coords_mapping_dict, True, MAPBOX_API_KEY)

[[121.039514, 14.653739], [121.028541, 14.689886], [121.020637, 14.69875]]
['', 'Mindanao Avenue', 'King Michael']
[array([121.0405753 ,  14.65583662]), array([121.02851216,  14.68985645]), array([121.02047848,  14.69872832])]
[[121.039514, 14.653739], [121.028541, 14.689886], [121.020637, 14.69875]]
['', 'Mindanao Avenue', 'King Michael']
[array([121.0405753 ,  14.65583662]), array([121.02851216,  14.68985645]), array([121.02047848,  14.69872832])]
[[121.028541, 14.689886], [121.039514, 14.653739], [121.020637, 14.69875]]
['Mindanao Avenue', '', 'King Michael']
[array([121.02851216,  14.68985645]), array([121.0405753 ,  14.65583662]), array([121.02047848,  14.69872832])]
[[121.028541, 14.689886], [121.020637, 14.69875], [121.039514, 14.653739]]
['Mindanao Avenue', 'King Michael', '']
[array([121.02851216,  14.68985645]), array([121.02047848,  14.69872832]), array([121.0405753 ,  14.65583662])]
[[121.028541, 14.689886], [121.020637, 14.69875], [121.039514, 14.653739]]
['Mindanao Avenue

In [5]:
pd.DataFrame(temp)

,stop_id_list,jeep_ids_list,jeep_locations_list,jeep_arrival_times_list
0,AL1,"[2, 0, 1]","[, Mindanao Avenue, King Michael]","[3069.8, 3971.7, 4584.4]"
1,AL2,"[2, 0, 1]","[, Mindanao Avenue, King Michael]","[5740.8, 6684.6, 7297.3]"
2,AL3,"[0, 2, 1]","[Mindanao Avenue, , King Michael]","[12265.3, 11321.5, 12878.0]"
3,AL4,"[0, 1, 2]","[Mindanao Avenue, King Michael, ]","[12394.8, 13007.5, 11451.0]"
4,AL5,"[0, 1, 2]","[Mindanao Avenue, King Michael, ]","[12394.8, 13007.5, 11451.0]"


In [11]:
x = np.array([[1,2],[3,4],[4,5]])

In [13]:
x[[0,2]]

array([[1, 2],
       [4, 5]])

In [6]:
query_time_to_stop(
    jeep_ids = ["ID1","ID2"], 
    stop_ids = ["AL1", "AL2"],
    stop_coords_mapping_dict = stop_coords_mapping_dict,
    use_mapbox = True,
    MAPBOX_API_KEY = MAPBOX_API_KEY
)


"list_of_times": [duration_matrix[:,i] for i in range(duration_matrix.shape[1])] # Gives the time from jeep i to stop j

,stop_id,list_of_jeeps,list_of_times
0,AL1,"[ID1, ID2]","[3550.1, 3882.0]"
1,AL2,"[ID1, ID2]","[6257.8, 6594.9]"
